# Convert all windows in `a-bam-p` to SMIRNOFF99Frosst and prepare the AMBER input files for running a test on TSCC

In [9]:
%load_ext autoreload
%autoreload 2

import os as os
import glob as glob
from utils import load_mol2, load_pdb, map_atoms, map_residues
from process import convert_parameters

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
base_path = 'a-bam-p/'
directories = []
for window in range(0, 14):
    directories.append('a' + f'{window:02d}')
for window in range(0, 1):
    directories.append('r' + f'{window:02d}')
for window in range(0, 46):
    directories.append('u' + f'{window:02d}')

In [13]:
reference_mol = load_mol2(base_path + 'a00/' + 'generated/' + 'reference.mol2')
target_mol = load_mol2(base_path + 'a00/' + 'generated/' + 'target.mol2')
atom_mapping = map_atoms(reference_mol, target_mol)

reference_mol = load_pdb(base_path + 'a00/' + 'generated/' + 'reference.pdb')
target_mol = load_pdb(base_path + 'a00/' + 'generated/' + 'target.pdb')
residue_mapping = map_residues(atom_mapping, reference_mol, target_mol)

Determining mapping...


In [79]:
import logging
from importlib import reload
reload(logging)

<module 'logging' from '/home/dslochower/data/applications/anaconda3/envs/openforcefield/lib/python3.6/logging/__init__.py'>

In [80]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(
    format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p')

In [ ]:
FORMAT = "%(asctime)-15s %(clientip)s %(user)-8s %(message)s"
logging.basicConfig(format=FORMAT)

In [81]:
logging.warning('hi')

2018-02-28 08:29:34 AM hi


In [68]:
for directory in directories:
    if not os.path.exists(base_path + directory + '/generated/'):
        os.makedirs(base_path + directory + '/generated/')
    print(f'\n{directory}')
    convert_parameters(source_directory=base_path + directory + '/original/',
                       source_crd='full.crds',
                       source_top='full.hmr.topo',
                       destination_directory=base_path + directory + '/generated/',
                       destination_crd='smirnoff.inpcrd',
                       destination_top='smirnoff.prmtop',
                       host_resname='MGO',
                       guest_resname='BAM',
                       atom_mapping=atom_mapping,
                       residue_mapping=residue_mapping)

INFO:root:Creating a-bam-p/a00/original/full.crds with CONECT records...
INFO:root:Pruning water-water CONECT records...



a00


INFO:root:Splitting topology into components...
INFO:root:Creating a combined topology for the host and guest molecules...
INFO:root:Writing a `mol2` for the host molecule...
INFO:root:Converting a-bam-p/a00/generated/MGO.mol2 to SYBYL atom types via Antechamber...
INFO:root:Loading a-bam-p/a00/generated/MGO-sybyl.mol2...
INFO:root:Loading a-bam-p/a00/original/bam.mol2...
INFO:root:Checking all atoms have unique names...
INFO:root:Checking all atoms have unique names...
INFO:root:Checking structure for bonds >4 A...
INFO:root:Extracting water and ions from a-bam-p/a00/original/full.hmr.topo...
INFO:root:Creating parameters for the waters and ions...
INFO:root:Writing a `frcmod` file for dummy atoms...
INFO:root:Writing a `mol2` file for dummy atoms...
INFO:root:Writing AMBER input file using residue mapping...
INFO:root:Writing AMBER input file using residue mapping...
INFO:root:Repartitioning hydrogen mass...
INFO:root:Writing AMBER input file using residue mapping...
INFO:root:Repart


a01


INFO:root:Splitting topology into components...
INFO:root:Creating a combined topology for the host and guest molecules...
INFO:root:Writing a `mol2` for the host molecule...


KeyboardInterrupt: 

In [56]:
minimization_engine   = '$AMBERHOME/bin/pmemd.cuda'
thermalization_engine = '$AMBERHOME/bin/pmemd.cuda'
equilibration_engine  = '$AMBERHOME/bin/pmemd.cuda'
production_engine     = '$AMBERHOME/bin/pmemd.cuda'


for directory in directories:
    if not os.path.exists(base_path + directory + '/generated/'):
        os.makedirs(base_path + directory + '/generated/')
    print(f'\n{directory}')
    with open(base_path + directory + '/generated/' + 'md.sh', 'w') as file:
        file.write('#!/usr/bin/env bash\n')
        # file.write('source $AMBERHOME/amber.sh\n')
        
        # file.write(f'{minimization_engine} -O -p smirnoff.prmtop -ref smirnoff.inpcrd -c smirnoff.inpcrd -i mini.in '
        #            '-o mini.out -r mini.rst -inf /dev/null\n')
        # file.write(f'{thermalization_engine} -O -p smirnoff.prmtop -ref smirnoff.inpcrd -c mini.rst -i therm1.in '
        #            '-o therm1.out -r therm1.rst -inf therm1.mdinfo\n')
        # file.write(f'{thermalization_engine} -O -p smirnoff.prmtop -ref smirnoff.inpcrd -c therm1.rst -i therm2.in '
        #            '-o therm2.out -r therm2.rst -inf therm2.mdinfo\n')
        # file.write('cp therm2.rst eqnpt.rst\n')
        # file.write('\n')
        # for iteration in range(1, 101):
            # file.write(f'{equilibration_engine} -O -p smirnoff.prmtop -ref smirnoff.inpcrd -c eqnpt.rst -i eqnpt.in '
            #            f'-o eqnpt.{iteration:02d}.out -r eqnpt.{iteration:02d}.rst -inf /dev/null >& eqnpt.log \n')
            # file.write(f'cat eqnpt.{iteration:02d}.out >> eqnpt.all.out\n')
            # file.write(f'cp eqnpt.{iteration:02d}.rst eqnpt.rst\n')
            # file.write('\n')

        # file.write('cp eqnpt.100.rst md.000.rst\n')
        file.write('\n')
        for iteration in range(1, 2):
            file.write(f'{production_engine} -O -p smirnoff.prmtop -ref smirnoff.inpcrd -c md.{(iteration-1):03d}.rst -i mdin '
                       f'-o mdout.{iteration:03d} -r md.{iteration:03d}.rst -x traj.{iteration:03d}.nc -inf mdinfo.{iteration:03d} -e mden.{iteration:03d} \n')
            file.write('\n')


a00

a01

a02

a03

a04

a05

a06

a07

a08

a09

a10

a11

a12

a13

r00

u00

u01

u02

u03

u04

u05

u06

u07

u08

u09

u10

u11

u12

u13

u14

u15

u16

u17

u18

u19

u20

u21

u22

u23

u24

u25

u26

u27

u28

u29

u30

u31

u32

u33

u34

u35

u36

u37

u38

u39

u40

u41

u42

u43

u44

u45


In [57]:
for directory in directories:
    if not os.path.exists(base_path + directory + '/generated/'):
        os.makedirs(base_path + directory + '/generated/')
    print(f'\n{directory}')
    with open(base_path + directory + '/generated/' + 'tscc.sh', 'w') as file:

        string = \
    f'''#!/bin/bash
#PBS -l walltime=2:00:00,nodes=1:ppn=3:gpu -q home-mgilson
#PBS -j oe -r n -m a -M slochower@gmail.com
#PBS -N {directory}

module swap intel gnu
module load cuda
module load python
module load scipy/2.7

PATH="/home/henrikse/gnu44-bin:/opt/mvapich2/gnu/ib/bin:$PATH"
CUDA_HOME="/opt/cuda/7.5.18"
LD_LIBRARY_PATH="${{CUDA_HOME}}/lib64:/opt/mvapich2/gnu/ib/lib:$LD_LIBRARY_PATH"
MV2_ENABLE_AFFINITY=0
SCRDIR=/oasis/tscc/scratch/davids4/a-bam-p/{directory}/
source /home/henrikse/amber16gnu/tsccamber.sh

# Synchronize all the files in the current directory with the scratch directory
mkdir -p $SCRDIR
rsync -avL $PBS_O_WORKDIR $SCRDIR

# Run the super duper python script
cd $SCRDIR/generated/
bash md.sh
'''
        file.write(string)


a00

a01

a02

a03

a04

a05

a06

a07

a08

a09

a10

a11

a12

a13

r00

u00

u01

u02

u03

u04

u05

u06

u07

u08

u09

u10

u11

u12

u13

u14

u15

u16

u17

u18

u19

u20

u21

u22

u23

u24

u25

u26

u27

u28

u29

u30

u31

u32

u33

u34

u35

u36

u37

u38

u39

u40

u41

u42

u43

u44

u45


In [54]:
with open(base_path + 'submit_all_jobs.sh', 'w') as file:
    file.write('#!/bin/bash\n')
    for directory in directories:
        file.write(f'cd /home/davids4/projects/smirnoff-host-guest/a-bam-p/{directory}/generated/\n')
        file.write('qsub tscc.sh\n')
        file.write('cd ../../\n')

Use local scratch (/state/partition1/$USER/$PBS_JOBID) if your job writes a lot of files from each task. The local scratch filesystem is purged at the end of each job, so you will need to copy out files that you want to retain after the job completes.
